In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import DataFrame, concat, read_csv
from arviz import hdi
sns.set_theme(style='white', context='notebook', font_scale=1.33)

## Section 1: Factor Loadings

### Model 1: Unidimensional model of maltreatment

#### Model parameters

In [2]:
## Define I/O parameters.
studies = ['teicher2015','tuominen2022']

## Main loop.
for i, study in enumerate(studies):
    if i: print('')
    print(f'study = {study}')

    ## Load samples.
    samples = read_csv(os.path.join('stan_results', study, 'grmq_m1.tsv.gz'), sep='\t', compression='gzip')
    
    for param in ['alpha','lambda']:
        
        ## Extract parameters.
        arr = samples.filter(regex=param).values
        
        ## Summarize across items.
        mu = arr.mean(axis=1).mean(); lb, ub = hdi(arr.mean(axis=1), hdi_prob=0.95)
        print('%s_mu:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

        ## Summarize least discriminating.
        ix = np.argmin(arr.mean(axis=0))
        mu = arr[:,ix].mean(); lb, ub = hdi(arr[:,ix], hdi_prob=0.95)
        print('%s_min:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

        ## Summarize most discriminating.
        ix = np.argmax(arr.mean(axis=0))
        mu = arr[:,ix].mean(); lb, ub = hdi(arr[:,ix], hdi_prob=0.95)
        print('%s_max:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

study = teicher2015
alpha_mu:	1.357 [1.295, 1.414]
alpha_min:	0.506 [0.067, 0.901]
alpha_max:	2.944 [2.505, 3.392]
lambda_mu:	0.597 [0.579, 0.614]
lambda_min:	0.279 [0.039, 0.468]
lambda_max:	0.864 [0.829, 0.895]

study = tuominen2022
alpha_mu:	1.349 [1.272, 1.422]
alpha_min:	0.657 [0.443, 0.864]
alpha_max:	2.407 [1.982, 2.873]
lambda_mu:	0.600 [0.578, 0.621]
lambda_min:	0.359 [0.256, 0.457]
lambda_max:	0.814 [0.763, 0.863]


#### Proportion of strong loadings

In [3]:
## Define I/O parameters.
studies = ['teicher2015','tuominen2022']

## Main loop.
for i, study in enumerate(studies):

    ## Load samples.
    samples = read_csv(os.path.join('stan_results', study, 'grmq_m1.tsv.gz'), sep='\t', compression='gzip')
            
    ## Extract parameters.
    arr = samples.filter(regex='lambda').values
    
    ## Compute proportion of strong loadings.
    prop = (arr > 0.5).mean(axis=1)
    mu = prop.mean(); lb, ub = hdi(prop, hdi_prob=0.95)
    
    ## Summarize information.
    print('%s:\t%0.3f [%0.3f, %0.3f]' %(study, mu, lb, ub))

teicher2015:	0.802 [0.718, 0.872]
tuominen2022:	0.800 [0.692, 0.872]


#### Comparison across datasets

In [4]:
from scipy.stats import spearmanr


for param in ['alpha','lambda']:

    ## Load item parameters (teicher2015).
    a1 = read_csv(os.path.join('stan_results', 'teicher2015', 'grmq_m1.tsv.gz'), sep='\t', 
                  compression='gzip').filter(regex=param).values

    ## Load item parameters (tuominen2022).
    a2 = read_csv(os.path.join('stan_results', 'tuominen2022', 'grmq_m1.tsv.gz'), sep='\t', 
                  compression='gzip').filter(regex=param).values

    ## Compute rank correlation.
    corr = np.array([spearmanr(i,j)[0] for i, j in zip(a1, a2)])
    mu = corr.mean(); lb, ub = hdi(corr, hdi_prob=0.95)
    print('%s:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

alpha:	0.565 [0.414, 0.711]
lambda:	0.565 [0.414, 0.711]


#### Eigenvalues

In [5]:
## Load polychoric correlation matrix.
poly = read_csv(os.path.join('stan_results', 'polychoric.csv'))

for study in ['teicher2015', 'tuominen2022']:
    corr = poly.pivot_table(study, 'k1', 'k2').values
    _, eig, _ = np.linalg.svd(corr)
    print(eig.round(2)[:6])

[14.62  3.77  2.64  2.11  1.79  1.35]
[13.27  4.43  2.48  2.04  1.81  1.56]


### Model 2: Bifactor model (w/ 10 specific factors)

#### Model parameters (general factor)

In [6]:
## Define I/O parameters.
studies = ['teicher2015','tuominen2022']

## Main loop.
for i, study in enumerate(studies):
    if i: print('')
    print(f'study = {study}')

    ## Load samples.
    samples = read_csv(os.path.join('stan_results', study, 'grmq_m2.tsv.gz'), sep='\t', compression='gzip')
    
    for param in ['alpha','lambda']:
        
        ## Extract parameters.
        arr = samples.filter(regex=f'{param}\[[0-9]*,1\]').values
        
        ## Summarize across items.
        mu = arr.mean(axis=1).mean(); lb, ub = hdi(arr.mean(axis=1), hdi_prob=0.95)
        print('%s_mu:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

        ## Summarize least discriminating.
        ix = np.argmin(arr.mean(axis=0))
        mu = arr[:,ix].mean(); lb, ub = hdi(arr[:,ix], hdi_prob=0.95)
        print('%s_min:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

        ## Summarize most discriminating.
        ix = np.argmax(arr.mean(axis=0))
        mu = arr[:,ix].mean(); lb, ub = hdi(arr[:,ix], hdi_prob=0.95)
        print('%s_max:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

study = teicher2015
alpha_mu:	1.503 [1.433, 1.579]
alpha_min:	0.470 [0.000, 0.830]
alpha_max:	4.248 [3.464, 4.996]
lambda_mu:	0.575 [0.556, 0.593]
lambda_min:	0.235 [0.000, 0.412]
lambda_max:	0.864 [0.829, 0.898]

study = tuominen2022
alpha_mu:	1.524 [1.434, 1.614]
alpha_min:	0.686 [0.430, 0.929]
alpha_max:	3.510 [2.364, 4.692]
lambda_mu:	0.580 [0.560, 0.601]
lambda_min:	0.299 [0.205, 0.393]
lambda_max:	0.827 [0.770, 0.877]


#### Model parameters (specific factors)

In [7]:
## Define I/O parameters.
studies = ['teicher2015','tuominen2022']

## Main loop.
for i, study in enumerate(studies):
    if i: print('')
    print(f'study = {study}')

    ## Load samples.
    samples = read_csv(os.path.join('stan_results', study, 'grmq_m2.tsv.gz'), sep='\t', compression='gzip')
    
    for param in ['alpha','lambda']:
                
        ## Extract parameters.
        arr = samples.filter(regex=f'{param}\[[0-9]*,[^1]|lambda\[[0-9]*,1[0-2]').values
        
        ## Summarize across items.
        mu = arr.mean(axis=1).mean(); lb, ub = hdi(arr.mean(axis=1), hdi_prob=0.95)
        print('%s_mu:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

        ## Summarize least discriminating.
        ix = np.argmin(arr.mean(axis=0))
        mu = arr[:,ix].mean(); lb, ub = hdi(arr[:,ix], hdi_prob=0.95)
        print('%s_min:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

        ## Summarize most discriminating.
        ix = np.argmax(arr.mean(axis=0))
        mu = arr[:,ix].mean(); lb, ub = hdi(arr[:,ix], hdi_prob=0.95)
        print('%s_max:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

study = teicher2015
alpha_mu:	0.804 [0.738, 0.873]
alpha_min:	0.035 [0.000, 0.113]
alpha_max:	2.916 [2.325, 3.589]
lambda_mu:	0.313 [0.285, 0.341]
lambda_min:	0.014 [0.000, 0.046]
lambda_max:	0.797 [0.743, 0.851]

study = tuominen2022
alpha_mu:	0.903 [0.821, 0.977]
alpha_min:	0.090 [0.000, 0.240]
alpha_max:	3.135 [1.822, 4.456]
lambda_mu:	0.343 [0.315, 0.369]
lambda_min:	0.043 [0.000, 0.138]
lambda_max:	0.698 [0.606, 0.788]


#### Relative parameter bias

In [8]:
## Define I/O parameters.
studies = ['teicher2015','tuominen2022']

## Main loop.
for i, study in enumerate(studies):
    
    l1 = read_csv(os.path.join('stan_results', study, 'grmq_m1.tsv.gz'), sep='\t', 
                  compression='gzip').filter(regex=f'lambda\[[0-9]*,1\]').values
    l2 = read_csv(os.path.join('stan_results', study, 'grmq_m2.tsv.gz'), sep='\t', 
                  compression='gzip').filter(regex=f'lambda\[[0-9]*,1\]').values
    
    ## Compute relative bias.
    bias = np.median((l1 - l2) / l2, axis=1)
    mu = np.median(bias); lb, ub = hdi(bias, hdi_prob=0.95)
    print('%s:\t%0.3f [%0.3f, %0.3f]' %(study, mu, lb, ub))

teicher2015:	0.021 [-0.024, 0.066]
tuominen2022:	0.027 [-0.026, 0.080]


### Model 3: Bifactor S-1 model (threat-deprivation)

#### Model parameters (general factor)

In [9]:
## Define I/O parameters.
studies = ['teicher2015','tuominen2022']

## Main loop.
for i, study in enumerate(studies):
    if i: print('')
    print(f'study = {study}')

    ## Load samples.
    samples = read_csv(os.path.join('stan_results', study, 'grmq_m3.tsv.gz'), sep='\t', compression='gzip')
    
    for param in ['alpha','lambda']:
        
        ## Extract parameters.
        arr = samples.filter(regex=f'{param}\[[0-9]*,1\]').values
        
        ## Summarize across items.
        mu = arr.mean(axis=1).mean(); lb, ub = hdi(arr.mean(axis=1), hdi_prob=0.95)
        print('%s_mu:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

        ## Summarize least discriminating.
        ix = np.argmin(arr.mean(axis=0))
        mu = arr[:,ix].mean(); lb, ub = hdi(arr[:,ix], hdi_prob=0.95)
        print('%s_min:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

        ## Summarize most discriminating.
        ix = np.argmax(arr.mean(axis=0))
        mu = arr[:,ix].mean(); lb, ub = hdi(arr[:,ix], hdi_prob=0.95)
        print('%s_max:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

study = teicher2015
alpha_mu:	1.348 [1.285, 1.408]
alpha_min:	0.320 [0.000, 0.665]
alpha_max:	2.944 [2.485, 3.384]
lambda_mu:	0.577 [0.558, 0.594]
lambda_min:	0.150 [0.000, 0.310]
lambda_max:	0.864 [0.832, 0.899]

study = tuominen2022
alpha_mu:	1.408 [1.331, 1.494]
alpha_min:	0.735 [0.533, 0.952]
alpha_max:	4.055 [3.164, 4.905]
lambda_mu:	0.580 [0.558, 0.601]
lambda_min:	0.395 [0.299, 0.488]
lambda_max:	0.831 [0.780, 0.884]


#### Model parameters (specific factors)

In [10]:
## Define I/O parameters.
studies = ['teicher2015','tuominen2022']

## Main loop.
for i, study in enumerate(studies):
    if i: print('')
    print(f'study = {study}')

    ## Load samples.
    samples = read_csv(os.path.join('stan_results', study, 'grmq_m3.tsv.gz'), sep='\t', compression='gzip')
    
    for param in ['alpha','lambda']:
                
        ## Extract parameters.
        arr = samples.filter(regex=f'{param}\[[0-9]*,[^1]|lambda\[[0-9]*,1[0-2]').values
        
        ## Summarize across items.
        mu = arr.mean(axis=1).mean(); lb, ub = hdi(arr.mean(axis=1), hdi_prob=0.95)
        print('%s_mu:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

        ## Summarize least discriminating.
        ix = np.argmin(arr.mean(axis=0))
        mu = arr[:,ix].mean(); lb, ub = hdi(arr[:,ix], hdi_prob=0.95)
        print('%s_min:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

        ## Summarize most discriminating.
        ix = np.argmax(arr.mean(axis=0))
        mu = arr[:,ix].mean(); lb, ub = hdi(arr[:,ix], hdi_prob=0.95)
        print('%s_max:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

study = teicher2015
alpha_mu:	0.982 [0.865, 1.114]
alpha_min:	0.064 [0.000, 0.196]
alpha_max:	1.552 [1.207, 1.907]
lambda_mu:	0.410 [0.373, 0.454]
lambda_min:	0.030 [0.000, 0.091]
lambda_max:	0.617 [0.534, 0.699]

study = tuominen2022
alpha_mu:	1.497 [1.310, 1.684]
alpha_min:	0.197 [0.000, 0.407]
alpha_max:	4.080 [3.192, 4.976]
lambda_mu:	0.454 [0.409, 0.499]
lambda_min:	0.092 [0.000, 0.188]
lambda_max:	0.704 [0.625, 0.772]


#### Relative parameter bias

In [11]:
## Define I/O parameters.
studies = ['teicher2015','tuominen2022']

## Main loop.
for i, study in enumerate(studies):
    
    l1 = read_csv(os.path.join('stan_results', study, 'grmq_m1.tsv.gz'), sep='\t', 
                  compression='gzip').filter(regex=f'lambda\[[0-9]*,1\]').values
    l2 = read_csv(os.path.join('stan_results', study, 'grmq_m3.tsv.gz'), sep='\t', 
                  compression='gzip').filter(regex=f'lambda\[[0-9]*,1\]').values
    
    ## Compute relative bias.
    bias = np.median((l1 - l2) / l2, axis=1)
    mu = np.median(bias); lb, ub = hdi(bias, hdi_prob=0.95)
    print('%s:\t%0.3f [%0.3f, %0.3f]' %(study, mu, lb, ub))

teicher2015:	0.010 [-0.026, 0.049]
tuominen2022:	0.011 [-0.039, 0.067]


### Model 4: Bifactor S-1 model (parimonious)

#### Model parameters (general factor)

In [12]:
## Define I/O parameters.
studies = ['teicher2015','tuominen2022']

## Main loop.
for i, study in enumerate(studies):
    if i: print('')
    print(f'study = {study}')

    ## Load samples.
    samples = read_csv(os.path.join('stan_results', study, 'grmq_m4.tsv.gz'), sep='\t', compression='gzip')
    
    for param in ['alpha','lambda']:
        
        ## Extract parameters.
        arr = samples.filter(regex=f'{param}\[[0-9]*,1\]').values
        
        ## Summarize across items.
        mu = arr.mean(axis=1).mean(); lb, ub = hdi(arr.mean(axis=1), hdi_prob=0.95)
        print('%s_mu:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

        ## Summarize least discriminating.
        ix = np.argmin(arr.mean(axis=0))
        mu = arr[:,ix].mean(); lb, ub = hdi(arr[:,ix], hdi_prob=0.95)
        print('%s_min:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

        ## Summarize most discriminating.
        ix = np.argmax(arr.mean(axis=0))
        mu = arr[:,ix].mean(); lb, ub = hdi(arr[:,ix], hdi_prob=0.95)
        print('%s_max:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

study = teicher2015
alpha_mu:	1.387 [1.320, 1.447]
alpha_min:	0.341 [0.000, 0.684]
alpha_max:	2.937 [2.494, 3.440]
lambda_mu:	0.563 [0.545, 0.580]
lambda_min:	0.158 [0.000, 0.311]
lambda_max:	0.864 [0.828, 0.898]

study = tuominen2022
alpha_mu:	1.440 [1.356, 1.520]
alpha_min:	0.657 [0.402, 0.887]
alpha_max:	4.175 [3.374, 4.988]
lambda_mu:	0.572 [0.550, 0.593]
lambda_min:	0.298 [0.199, 0.393]
lambda_max:	0.852 [0.803, 0.896]


#### Model parameters (specific factors)

In [13]:
## Define I/O parameters.
studies = ['teicher2015','tuominen2022']

## Main loop.
for i, study in enumerate(studies):
    if i: print('')
    print(f'study = {study}')

    ## Load samples.
    samples = read_csv(os.path.join('stan_results', study, 'grmq_m4.tsv.gz'), sep='\t', compression='gzip')
    
    for param in ['alpha','lambda']:
                
        ## Extract parameters.
        arr = samples.filter(regex=f'{param}\[[0-9]*,[^1]|lambda\[[0-9]*,1[0-2]').values
        
        ## Summarize across items.
        mu = arr.mean(axis=1).mean(); lb, ub = hdi(arr.mean(axis=1), hdi_prob=0.95)
        print('%s_mu:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

        ## Summarize least discriminating.
        ix = np.argmin(arr.mean(axis=0))
        mu = arr[:,ix].mean(); lb, ub = hdi(arr[:,ix], hdi_prob=0.95)
        print('%s_min:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

        ## Summarize most discriminating.
        ix = np.argmax(arr.mean(axis=0))
        mu = arr[:,ix].mean(); lb, ub = hdi(arr[:,ix], hdi_prob=0.95)
        print('%s_max:\t%0.3f [%0.3f, %0.3f]' %(param, mu, lb, ub))

study = teicher2015
alpha_mu:	1.402 [1.306, 1.495]
alpha_min:	0.655 [0.419, 0.918]
alpha_max:	2.746 [2.216, 3.274]
lambda_mu:	0.550 [0.522, 0.576]
lambda_min:	0.327 [0.214, 0.441]
lambda_max:	0.789 [0.737, 0.840]

study = tuominen2022
alpha_mu:	1.526 [1.388, 1.658]
alpha_min:	0.363 [0.020, 0.646]
alpha_max:	3.744 [2.799, 4.722]
lambda_mu:	0.526 [0.492, 0.556]
lambda_min:	0.183 [0.019, 0.326]
lambda_max:	0.684 [0.588, 0.773]


#### Relative parameter bias

In [14]:
## Define I/O parameters.
studies = ['teicher2015','tuominen2022']

## Main loop.
for i, study in enumerate(studies):
    
    l1 = read_csv(os.path.join('stan_results', study, 'grmq_m1.tsv.gz'), sep='\t', 
                  compression='gzip').filter(regex=f'lambda\[[0-9]*,1\]').values
    l2 = read_csv(os.path.join('stan_results', study, 'grmq_m4.tsv.gz'), sep='\t', 
                  compression='gzip').filter(regex=f'lambda\[[0-9]*,1\]').values
    
    ## Compute relative bias.
    bias = np.median((l1 - l2) / l2, axis=1)
    mu = np.median(bias); lb, ub = hdi(bias, hdi_prob=0.95)
    print('%s:\t%0.3f [%0.3f, %0.3f]' %(study, mu, lb, ub))

teicher2015:	0.024 [-0.020, 0.074]
tuominen2022:	0.035 [-0.020, 0.100]


## Section 2: Variance Decomposition

In [15]:
## Load design data.
design = read_csv(os.path.join('data', 'design.csv'), index_col=0)

## Define locally dependent items.
ld = [[6,7,8], [9,10,11], [13,14], [15,16], [19,20], [21,22,23], [24,25], [33,34,35], [36,37]]
for ix in ld: design = design.drop(index=ix[1:])

### Model 2

In [16]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define I/O parameters.
studies = ['teicher2015', 'tuominen2022']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Restrict to columns of interest.
D = design[design.columns[:11]].copy()

stats = []
for study in studies:
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load Stan summary.
    summary = read_csv(os.path.join('stan_results', study, 'grmq_m2_summary.tsv'), sep='\t', index_col=0)
    
    ## Extract factor loadings.
    loadings = np.zeros_like(D).astype(float)
    for i, j in np.column_stack([np.where(D)]).T:
        loadings[i,j] = summary.loc[f'lambda[{i+1},{j+1}]','Mean']
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Coefficient omega hierachical.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
    ## Preallocate space.
    omega   = np.zeros(len(D.columns))
    omega_s = np.zeros(len(D.columns))
        
    ## Iterate over factors.
    for i, col in enumerate(D.columns):
        
        ## Restrict to items in group.
        L = loadings[D[col]==1]
        
        ## Compute squared sum of factor loadings.
        A = np.square(np.sum(L, axis=0))
        
        ## Compute sum of error variances.
        B = np.sum(1 - np.square(L).sum(axis=1))
        
        ## Compute total variance.
        C = np.sum(A) + B
        
        ## Compute coefficient omega.
        omega[i] = A.sum() / C
        
        ## Compute coefficient omega subscale.
        omega_s[i] = A[i] / C
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Explained common variance.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
    ## Compute sum of squares.
    ss = np.square(loadings).sum(axis=0)
    
    ## Compute explained common variance.
    ecv = ss / ss.sum()
    
    ## Compute PUC.
    A = len(D) * (len(D) - 1) / 2
    B = sum([sum(D[col]) * (sum(D[col])-1) / 2 for col in D.columns[1:]])
    puc = (A - B) / A
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### H-index
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Preallocate space.
    H = np.zeros(len(D.columns))
    
    ## Iterate over factors.
    for i, col in enumerate(D.columns):
        
        ## Compute squared loadings.
        s = np.square(loadings[:,i])
        
        ## Compute H-index.
        H[i] = 1. / (1 + 1 / np.sum(s / (1-s)))
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Convert to DataFrame.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    stats.append(DataFrame(dict(
        subscale = D.columns,
        study = np.repeat(study, D.columns.size),
        ecv = ecv,
        puc = puc,
        omega = omega,
        omega_s = omega_s,
        H = H
    )))
    
## Concatenate DataFrames.
stats = concat(stats).replace({'teicher2015':1, 'tuominen2022': 2})

## Convert to pivot table.
stats = stats.pivot_table(['omega','omega_s','ecv','puc','H'], 'subscale', 'study').round(3)
stats = stats.applymap(lambda x: '%0.3f' %x)
stats.loc[stats.index!='general',['ecv','puc']] = ''

## Re-organize rows.
index = ['general', 'VA', 'PA', 'NVEA', 'SA', 'EN', 'PN', 'WSV', 'WIPV', 'PeerVA', 'PeerPA']
stats = stats.loc[index]

## Re-organize columns.
cols = [(1,'puc'),(1,'ecv'),(1,'omega'),(1,'omega_s'),(1,'H'),
        (2,'ecv'),(2,'omega'),(2,'omega_s'),(2,'H')]
stats = stats.swaplevel(axis='columns')[cols]

## Display table.
stats

study         1                                   2                      
            puc    ecv  omega omega_s      H    ecv  omega omega_s      H
subscale                                                                 
general   0.912  0.716  0.963   0.924  0.964  0.697  0.965   0.924  0.961
VA                      0.909   0.069  0.273         0.893   0.162  0.478
PA                      0.590   0.037  0.052         0.595   0.148  0.194
NVEA                    0.848   0.136  0.525         0.827   0.117  0.424
SA                      0.710   0.059  0.134         0.749   0.290  0.452
EN                      0.715   0.162  0.304         0.874   0.203  0.542
PN                      0.800   0.217  0.479         0.812   0.114  0.284
WSV                     0.695   0.027  0.053         0.651   0.037  0.067
WIPV                    0.655   0.146  0.197         0.612   0.077  0.107
PeerVA                  0.878   0.621  0.818         0.853   0.565  0.766
PeerPA                  0.699   0.335  0.443         0.694   0.337  0.446

In [17]:
print(stats.to_latex())

\begin{tabular}{llllllllll}
\toprule
study & \multicolumn{5}{l}{1} & \multicolumn{4}{l}{2} \\
{} &    puc &    ecv &  omega & omega\_s &      H &    ecv &  omega & omega\_s &      H \\
subscale &        &        &        &         &        &        &        &         &        \\
\midrule
general  &  0.912 &  0.716 &  0.963 &   0.924 &  0.964 &  0.697 &  0.965 &   0.924 &  0.961 \\
VA       &        &        &  0.909 &   0.069 &  0.273 &        &  0.893 &   0.162 &  0.478 \\
PA       &        &        &  0.590 &   0.037 &  0.052 &        &  0.595 &   0.148 &  0.194 \\
NVEA     &        &        &  0.848 &   0.136 &  0.525 &        &  0.827 &   0.117 &  0.424 \\
SA       &        &        &  0.710 &   0.059 &  0.134 &        &  0.749 &   0.290 &  0.452 \\
EN       &        &        &  0.715 &   0.162 &  0.304 &        &  0.874 &   0.203 &  0.542 \\
PN       &        &        &  0.800 &   0.217 &  0.479 &        &  0.812 &   0.114 &  0.284 \\
WSV      &        &        &  0.695 &   0.027 

<ipython-input-17-eee063aaf1d3>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(stats.to_latex())


### Model 3

In [18]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define I/O parameters.
studies = ['teicher2015', 'tuominen2022']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Restrict to columns of interest.
D = design[['general','neglect']].copy()

stats = []
for study in studies:
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load Stan summary.
    summary = read_csv(os.path.join('stan_results', study, 'grmq_m3_summary.tsv'), sep='\t', index_col=0)
    
    ## Extract factor loadings.
    loadings = np.zeros_like(D).astype(float)
    for i, j in np.column_stack([np.where(D)]).T:
        loadings[i,j] = summary.loc[f'lambda[{i+1},{j+1}]','Mean']
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Coefficient omega hierachical.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
    ## Preallocate space.
    omega   = np.zeros(len(D.columns))
    omega_s = np.zeros(len(D.columns))
        
    ## Iterate over factors.
    for i, col in enumerate(D.columns):
        
        ## Restrict to items in group.
        L = loadings[D[col]==1]
        
        ## Compute squared sum of factor loadings.
        A = np.square(np.sum(L, axis=0))
        
        ## Compute sum of error variances.
        B = np.sum(1 - np.square(L).sum(axis=1))
        
        ## Compute total variance.
        C = np.sum(A) + B
        
        ## Compute coefficient omega.
        omega[i] = A.sum() / C
        
        ## Compute coefficient omega subscale.
        omega_s[i] = A[i] / C
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Explained common variance.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
    ## Compute sum of squares.
    ss = np.square(loadings).sum(axis=0)
    
    ## Compute explained common variance.
    ecv = ss / ss.sum()
    
    ## Compute PUC.
    A = len(D) * (len(D) - 1) / 2
    B = sum([sum(D[col]) * (sum(D[col])-1) / 2 for col in D.columns[1:]])
    puc = (A - B) / A
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### H-index
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Preallocate space.
    H = np.zeros(len(D.columns))
    
    ## Iterate over factors.
    for i, col in enumerate(D.columns):
        
        ## Compute squared loadings.
        s = np.square(loadings[:,i])
        
        ## Compute H-index.
        H[i] = 1. / (1 + 1 / np.sum(s / (1-s)))
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Convert to DataFrame.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    stats.append(DataFrame(dict(
        subscale = D.columns,
        study = np.repeat(study, D.columns.size),
        ecv = ecv,
        puc = puc,
        omega = omega,
        omega_s = omega_s,
        H = H
    )))
    
## Concatenate DataFrames.
stats = concat(stats).replace({'teicher2015':1, 'tuominen2022': 2})

## Convert to pivot table.
stats = stats.pivot_table(['omega','omega_s','ecv','puc','H'], 'subscale', 'study').round(3)
stats = stats.applymap(lambda x: '%0.3f' %x)
stats.loc[stats.index!='general',['ecv','puc']] = ''

## Re-organize rows.
index = D.columns
stats = stats.loc[index]

## Re-organize columns.
cols = [(1,'puc'),(1,'ecv'),(1,'omega'),(1,'omega_s'),(1,'H'),
        (2,'ecv'),(2,'omega'),(2,'omega_s'),(2,'H')]
stats = stats.swaplevel(axis='columns')[cols]

## Display table.
stats

study        1                                   2                      
           puc    ecv  omega omega_s      H    ecv  omega omega_s      H
general  0.939  0.864  0.958   0.928  0.964  0.841  0.959   0.922  0.959
neglect                0.877   0.384  0.766         0.921   0.375  0.814

In [19]:
print(stats.to_latex())

<ipython-input-19-eee063aaf1d3>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(stats.to_latex())


\begin{tabular}{llllllllll}
\toprule
study & \multicolumn{5}{l}{1} & \multicolumn{4}{l}{2} \\
{} &    puc &    ecv &  omega & omega\_s &      H &    ecv &  omega & omega\_s &      H \\
\midrule
general &  0.939 &  0.864 &  0.958 &   0.928 &  0.964 &  0.841 &  0.959 &   0.922 &  0.959 \\
neglect &        &        &  0.877 &   0.384 &  0.766 &        &  0.921 &   0.375 &  0.814 \\
\bottomrule
\end{tabular}



### 2.3 Model 4

In [20]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Define parameters.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Define I/O parameters.
studies = ['teicher2015', 'tuominen2022']

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

## Restrict to columns of interest.
D = design[['general','peer','reverse']].copy()

stats = []
for study in studies:
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

    ## Load Stan summary.
    summary = read_csv(os.path.join('stan_results', study, 'grmq_m4_summary.tsv'), sep='\t', index_col=0)
    
    ## Extract factor loadings.
    loadings = np.zeros_like(D).astype(float)
    for i, j in np.column_stack([np.where(D)]).T:
        loadings[i,j] = summary.loc[f'lambda[{i+1},{j+1}]','Mean']
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Coefficient omega hierachical.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
    ## Preallocate space.
    omega   = np.zeros(len(D.columns))
    omega_s = np.zeros(len(D.columns))
        
    ## Iterate over factors.
    for i, col in enumerate(D.columns):
        
        ## Restrict to items in group.
        L = loadings[D[col]==1]
        
        ## Compute squared sum of factor loadings.
        A = np.square(np.sum(L, axis=0))
        
        ## Compute sum of error variances.
        B = np.sum(1 - np.square(L).sum(axis=1))
        
        ## Compute total variance.
        C = np.sum(A) + B
        
        ## Compute coefficient omega.
        omega[i] = A.sum() / C
        
        ## Compute coefficient omega subscale.
        omega_s[i] = A[i] / C
        
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Explained common variance.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
        
    ## Compute sum of squares.
    ss = np.square(loadings).sum(axis=0)
    
    ## Compute explained common variance.
    ecv = ss / ss.sum()
    
    ## Compute PUC.
    A = len(D) * (len(D) - 1) / 2
    B = sum([sum(D[col]) * (sum(D[col])-1) / 2 for col in D.columns[1:]])
    puc = (A - B) / A
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### H-index
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    ## Preallocate space.
    H = np.zeros(len(D.columns))
    
    ## Iterate over factors.
    for i, col in enumerate(D.columns):
        
        ## Compute squared loadings.
        s = np.square(loadings[:,i])
        
        ## Compute H-index.
        H[i] = 1. / (1 + 1 / np.sum(s / (1-s)))
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Convert to DataFrame.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    
    stats.append(DataFrame(dict(
        subscale = D.columns,
        study = np.repeat(study, D.columns.size),
        ecv = ecv,
        puc = puc,
        omega = omega,
        omega_s = omega_s,
        H = H
    )))
    
## Concatenate DataFrames.
stats = concat(stats).replace({'teicher2015':1, 'tuominen2022': 2})

## Convert to pivot table.
stats = stats.pivot_table(['omega','omega_s','ecv','puc','H'], 'subscale', 'study').round(3)
stats = stats.applymap(lambda x: '%0.3f' %x)
stats.loc[stats.index!='general',['ecv','puc']] = ''

## Re-organize rows.
index = D.columns
stats = stats.loc[index]

## Re-organize columns.
cols = [(1,'puc'),(1,'ecv'),(1,'omega'),(1,'omega_s'),(1,'H'),
        (2,'ecv'),(2,'omega'),(2,'omega_s'),(2,'H')]
stats = stats.swaplevel(axis='columns')[cols]

## Display table.
stats

study        1                                   2                      
           puc    ecv  omega omega_s      H    ecv  omega omega_s      H
general  0.931  0.738  0.961   0.896  0.965  0.751  0.961   0.904  0.960
peer                   0.889   0.569  0.842         0.868   0.493  0.781
reverse                0.839   0.584  0.739         0.915   0.498  0.773

In [21]:
print(stats.to_latex())

\begin{tabular}{llllllllll}
\toprule
study & \multicolumn{5}{l}{1} & \multicolumn{4}{l}{2} \\
{} &    puc &    ecv &  omega & omega\_s &      H &    ecv &  omega & omega\_s &      H \\
\midrule
general &  0.931 &  0.738 &  0.961 &   0.896 &  0.965 &  0.751 &  0.961 &   0.904 &  0.960 \\
peer    &        &        &  0.889 &   0.569 &  0.842 &        &  0.868 &   0.493 &  0.781 \\
reverse &        &        &  0.839 &   0.584 &  0.739 &        &  0.915 &   0.498 &  0.773 \\
\bottomrule
\end{tabular}



<ipython-input-21-eee063aaf1d3>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(stats.to_latex())
